# **DeepLandforms - v2**

Author: giacomo.nodjoumi@hyranet.info - g.nodjoumi@jacobs-university.de

## DeepLandforms

With this notebook, users can use custom [YOLOv8](https://github.com/ultralytics/ultralytics) trained models for object detection and instance segmentation models on custom dataset of georeferenced images.
Results can be visualized directly in the noteboo using leafmap and WMS backend.

The output consist of a folder containing:
* Crop of the detections (georeferenced)
* Label file in COCO json format for each image for segmentation
* Label file in YOLO txt format for object detection
* Geopackage containing a single layer with image name, confidence leve, class.

## Usage

* Put or link the dataset into the **DeepLandforms** *.env* file
* Run docker-compose up
* Edit the *configs* section by editing the following parameters:

## Parameters
 ------------------------------------------------------------------
| **Parameter** | **Description** | **Example** |
| ---- | ---- | ---- |
| **data_dir** | local path of the data dir |  | /home/user/data |
| **model_path** | local path and name of the model  | /home/user/data/best.pt |
| **sam_checkpoint** | Segment Anything checkpoint for instance segmentation | /home/user/data/sam_vit_h_4b8939.pth |
| **dst_crs** | CRS of the final geopackage | provide as WKT or proj4 |
| **device** | device where to run the model | cuda or cpu |
------------------------------------------------------------------
Then just execute the notebook and monitor the training in **Tensorboard** container.

## Funding
*This study is within the Europlanet 2024 RI and EXPLORE project, and it has received funding from the European Union’s Horizon 2020 research and innovation programme under grant agreement No 871149 and No 101004214.*

In [1]:
from pyproj.crs import CRS
dst_crs = CRS.from_wkt('GEOGCRS["GCS_Mars_2000",DATUM["D_Mars_2000",ELLIPSOID["Mars_2000_IAU_IAG",3396190,169.894447223612,LENGTHUNIT["metre",1]]],PRIMEM["Reference_Meridian",0,ANGLEUNIT["degree",0.0174532925199433]],CS[ellipsoidal,2],AXIS["geodetic latitude (Lat)",north,ORDER[1],ANGLEUNIT["degree",0.0174532925199433]],AXIS["geodetic longitude (Lon)",east,ORDER[2],ANGLEUNIT["degree",0.0174532925199433]],USAGE[SCOPE["Not known."],AREA["World."],BBOX[-90,-180,90,180]],ID["ESRI",104905]]')
#dst_crs = CRS.from_wkt('PROJCS["Moon_2000_Equidistant_Cylindrical",GEOGCS["GCS_Moon_2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Reference_Meridian",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Equidistant_Cylindrical_Ellipsoidal"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],UNIT["Meter",1.0]]')

In [2]:
from ultralytics import YOLO
import geopandas as gpd
import json
import math
import numpy as np
import os
import pandas as pd
import rasterio as rio
from utils.utils import get_paths, window_calc, mask2shape, bboxes2df, bbox2points, box2geotiff, box2sam, PlotMap

In [5]:
test_dir = '/home/Giacomo/data/CTX-HiRISE-TEST_YOLOSAM/'
dst_dir = f"{test_dir}/detections"
os.makedirs(dst_dir, exist_ok=True)

In [6]:
model_path= '/home/Giacomo/DeepLandforms/YOLO/runs/detect/yolov8s_640_8_3m_28082023_4/weights/best.pt'
model = YOLO(model_path)

In [7]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
SamAutomaticMaskGenerator, SamPredictor
sam_checkpoint = "/home/Giacomo/data/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device="cuda")
predictor = SamPredictor(sam)

In [9]:
image_list = get_paths(test_dir, 'tiff')

In [10]:
cols = ['Image','Class','Conf']
geo_shape = gpd.GeoDataFrame(columns=cols)#, crs=img.crs)
geo_points = gpd.GeoDataFrame(columns=cols)#, crs=dst_crs)

In [11]:
image_list

['B11_013869_2027_XN_22N210W.tiff',
 'B11_013879_1720_XN_08S119W.tiff',
 'B12_014278_2053_XN_25N216W.tiff',
 'B05_011756_1734_XN_06S120W_H0_V0_resized_10.0m.tiff']

geo_points=gpd.read_file('/home/Giacomo/data/BC_n_SQCRP_n_CellSize_5_m_LIM_n_None_px_cog_n/detections/point_detections.gpkg')
geo_shape = gpd.read_file('/home/Giacomo/data/BC_n_SQCRP_n_CellSize_5_m_LIM_n_None_px_cog_n/detections/shape_detections.gpkg')

In [12]:
for file in image_list:
    try:
        image=f"{test_dir}/{file}"
        image_name, ext = os.path.splitext(os.path.basename(image))
        
        image_dir = os.path.dirname(image)
        img = rio.open(image)    
        aff = img.transform
        width = img.width
        height = img.height
        img_crs = img.crs
        results = model.predict(image, project="project",name="prediction", imgsz=640, conf=0.2)#, iou=0.3)#, half=True)  # predict on an imagesave_crop=True, save_txt=True, save_conf=True, 
        for result in results:#
            bboxes = np.array(result.boxes.data.cpu())
            xywh_bboxes = np.array(result.boxes.xywhn.data.cpu())
            classes = np.array(result.boxes.cls.data.cpu()).astype(int)
            confs = np.array(result.boxes.conf.data.cpu())
            shapes=[]
            for i, data in enumerate(bboxes):    
                bbox = data[0:4]
                conf = round(data[4],2)
                cls = result.names[data[5]]
                dst_name = box2geotiff(bbox, img,dst_dir, image_name, ext, cls, i)
    
                sam_gdf, shape_dict = box2sam(predictor, image, bbox, img_crs, conf, cls)
                shapes.append(shape_dict)
                if sam_gdf.crs != dst_crs:            
                    sam_gdf.to_crs(dst_crs, inplace=True)
                geo_shape=pd.concat([sam_gdf, geo_shape])
            
            yolo_fdf = bboxes2df(xywh_bboxes, classes,confs, cols=['x','y','w','h'])
            yolo_sdf=yolo_fdf[['Class','x','y','w','h']]#,'Conf']]
            csv_name = f"{dst_dir}/{image_name}.txt"
            yolo_sdf.to_csv(csv_name, header=False, sep=' ', index=False)        
            yolo_gdf = bbox2points(yolo_fdf, image)#xywh_bboxes, image_name,width, height,image)        
            if yolo_gdf.crs != dst_crs:            
                yolo_gdf.to_crs(dst_crs, inplace=True)
            geo_points=pd.concat([yolo_gdf, geo_points])
        
        label_dict = {
          "version": "5.2.1",
          "flags": {},
          "shapes": shapes,
          "imagePath": os.path.basename(image),
          "imageData": None,
          "imageHeight": height,
          "imageWidth": width
        }
        
        json_name = f"{dst_dir}/{image_name}.json"
        out_file = open(json_name, 'w')
        json.dump(label_dict,out_file,indent=2)
        out_file.close()      
    except Exception as e:
        print(e)
            
        
geo_points.reset_index(drop=True)
geo_points.crs=dst_crs
point_gpkg = f"{dst_dir}/point_detections.gpkg"
#geo_points.to_crs(dst_crs, inplace=True)
geo_points.to_file(point_gpkg, layer='PointDetections', driver="GPKG")


geo_shape.reset_index(drop=True)
geo_shape.crs=dst_crs
shape_gpkg = f"{dst_dir}/shape_detections.gpkg"
#geo_points.to_crs(dst_crs, inplace=True)
geo_shape.to_file(shape_gpkg, layer='ShapesDetections', driver="GPKG")


image 1/1 /home/Giacomo/data/CTX-HiRISE-TEST_YOLOSAM/B11_013869_2027_XN_22N210W.tiff: 640x224 (no detections), 44.5ms
Speed: 3.5ms preprocess, 44.5ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 224)

image 1/1 /home/Giacomo/data/CTX-HiRISE-TEST_YOLOSAM/B11_013879_1720_XN_08S119W.tiff: 640x416 1 Type-4, 43.6ms
Speed: 3.8ms preprocess, 43.6ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 416)

image 1/1 /home/Giacomo/data/CTX-HiRISE-TEST_YOLOSAM/B12_014278_2053_XN_25N216W.tiff: 640x160 (no detections), 41.7ms
Speed: 3.0ms preprocess, 41.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 160)

image 1/1 /home/Giacomo/data/CTX-HiRISE-TEST_YOLOSAM/B05_011756_1734_XN_06S120W_H0_V0_resized_10.0m.tiff: 640x512 1 Type-1, 47.0ms
Speed: 4.1ms preprocess, 47.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 512)


In [ ]:
map_select = PlotMap('Mars')
#geo_points.crs = "EPSG:4326"
#map_select.add_gdf(geo_points,layer_name='PointsDetections')

map_select

In [ ]:
aaa

In [ ]:
moon2000=CRS.from_wkt('GEOGCS["GCS_Moon_2000",DATUM["D_Moon_2000",SPHEROID["Moon_2000_IAU_IAG",1737400.0,0.0]],PRIMEM["Reference_Meridian",0.0],UNIT["Degree",0.0174532925199433]]')


In [ ]:
geo_points_crs = geo_points.copy()
#geo_points.crs = dst_crs#"EPSG:4326"
geo_points_crs=geo_points_crs.to_crs(dst_crs)
#geo_points_crs
geo_points_crs.crs ="EPSG:4326"
geo_points_crs.crs

In [ ]:
geo_shape_crs = geo_shape.copy()
#geo_shape_crs= geo_shape_crs.to_crs(moon2000)
geo_shape_crs.crs = "EPSG:4326"

map_select.add_gdf(geo_shape_crs,layer_name='Detections',fill_colors=["red", "green", "blue"])#,style = style,)
map_select.add_gdf(geo_points_crs,layer_name='PointsDetections',fill_colors=["red", "green", "blue"])#,style = style,)
map_select

In [ ]:

map_select

In [ ]:
geo_shape_crs = geo_shape.copy()

geo_shape_crs.crs = "EPSG:4326"
geo_shape= geo_shape.to_crs(moon2000)
map_select.add_gdf(geo_shape_crs,layer_name='Detections',fill_colors=["red", "green", "blue"])#,style = style,)
map_select

In [ ]:
geo_shape_crs

In [14]:
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction, visualize_object_predictions
from sahi.utils.cv import read_image_as_pil

In [29]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=model_path,
    confidence_threshold=0.8,
    device="cuda:0", # or 'cpu'
)

In [24]:
image_list

['B11_013869_2027_XN_22N210W.tiff',
 'B11_013879_1720_XN_08S119W.tiff',
 'B12_014278_2053_XN_25N216W.tiff',
 'B05_011756_1734_XN_06S120W_H0_V0_resized_10.0m.tiff']

In [32]:
image=f"{test_dir}/{image_list[0]}"
print(f"{image}")
result = get_sliced_prediction(
    image,
    detection_model,
    slice_height = 640,
    slice_width = 640,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2
)

/home/Giacomo/data/CTX-HiRISE-TEST_YOLOSAM//B11_013869_2027_XN_22N210W.tiff
Performing prediction on 616 number of slices.


In [33]:
result.export_visuals(export_dir=f"{test_dir}/results.jpg")

#Image("demo_data/prediction_visual.png")

In [ ]:
import cv2
from PIL import Image
import numpy as np
img = cv2.imread(image)
img_converted = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
numpydata = np.asarray(img_converted)
visualize_object_predictions(
    numpydata, 
    object_prediction_list = result.object_prediction_list,
    hide_labels = 1, 
    output_dir='./',
    file_name = 'result',
    export_format = 'png'
)
#Image.Image('demo_data/result.png')